In [10]:
#import sys
# !{sys.executable} -m pip install pymysql
# !{sys.executable} -m pip install ratelimiter
# !{sys.executable} -m pip install sqlalchemy

import pandas as pd
import requests
from ratelimiter import RateLimiter
from Location import Location


In [ ]:
ufos = pd.read_csv(r'/Users/kellan/Downloads/ufo-reports-master/csv-data/ufo-scrubbed-geocoded-time-standardized.csv')
ufos_canada = ufos[ufos['Country'] == 'ca']

ufos_canada.head()

As we can see, there are some problems with the location data. Let's delete the columns and recalculate

In [ ]:
ufos_canada.State.unique()
ufos_canada.drop(columns=['State', 'City', 'Country', 'Date added'], inplace =True)

Now, let's call the api using Location, and calculate our columns. The reverse geocoding is done through the locationiq.com API

In [ ]:

location_info = Location.Location()

with open("token.txt") as f:
    private_token = f.read()

location_columns = ['full_address', 'neighbourhood', 'city', 'province', 'country_code', 'country']
for col in location_columns:
    ufos_canada[col] = ''

ufos_canada['location_data_raw'] = ufos_canada.apply(lambda x: Location.make_request(x['Lat'], 
                                                                                     x['Lon'], 
                                                                                     private_token).return_raw(), axis=1)

location_columns = ['full_address', 'neighbourhood', 'city', 'province', 'country_code', 'country']
ufos_canada[location_columns] = ufos_canada.apply(lambda x: Location.get_all(x['location_data_raw']), axis=1)
ufos_canada.reset_index(inplace=True, drop=True)
ufos_canada.columns = ufos_canada.columns.str.title().str.strip()

ufos_canada.to_csv('canada_ufos_corrected.csv', index=False)

In [ ]:
ufos_canada

Next, we now will write the full dataset to a table, before breaking up the data set into separate provinces